# Moving Averge-based Trading Strategies

## Fetching Data from CryptoCompare

In [1]:
import requests
import json
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

from datetime import datetime
import time

# ----- User Settings 
#coins=['BTC', 'ETH','LTC','MKR','AAVE', 'BAL', 'UNI', 'LDO', 'MATIC','LINK']
coins=['BTC', 'ETH','LTC','LINK']

trading_fee = 0.5/100 
collectDataFrom = pd.Timestamp('2022-01-01')
trading_range = [+0.02,+2]
#buyAbove='sma7d'
#buyAbove='sma50d'
#buyAbove='sma200d'
buyAbove='sma200w'

cryptocompare_api_key='777db0605f662ba6667a0c8d385d317262bd221764d4a3b01dc7a629e9592a49'


# ----- User Settings -- end

coin_colors = {
    'BTC':'orange',
    'ETH':'grey',
    'AAVE':'purple',
    'LTC':'lightblue',
    'MKR':'lightgreen',
    'MATIC':'blue',
    'UNI':'pink',
    'LDO':'darkred',
    'BAL':'brown',
    'LQTY':'teal',
    'BIFI' :'lightgrey',
    'LINK' : 'darkblue'
}

initial_coin = 1
initial_usd = 0
limit=2000

coin_dfs=[]
for coin in coins:
    time_segments = []
    timestamp = pd.Timestamp.now()
    while collectDataFrom <= timestamp : 
        requestString = 'https://min-api.cryptocompare.com/data/v2/histoday?fsym={}&tsym=USD&limit={}&api_key={}&toTs={}'\
        .format(coin, limit, cryptocompare_api_key, timestamp.timestamp())
        print("Request : " + requestString)    
        response = requests.get(requestString).text 
        parsed = json.loads(response)
        segment_df = pd.DataFrame(parsed['Data']['Data'])
        segment_df['time'] = pd.to_datetime(segment_df['time'], unit='s')
        segment_df.set_index(['time'],inplace=True, verify_integrity = True, drop = True)
        segment_df.sort_index()

        #segment_df.drop_duplicates(inplace=True)
        timestamp = segment_df.first_valid_index() - pd.Timedelta(1, "d") # -1 day        
        #segment_df= segment_df.head(1)
        time_segments.append(segment_df)
          
    df = pd.concat(time_segments, axis=0)
    df = pd.concat({coin: df}, axis=1, names=["Coin", "Metrics"])
    df.drop_duplicates(inplace=True)
    df.drop(columns=['volumefrom','volumeto','conversionType','conversionSymbol'], level='Metrics', inplace=True)
    coin_dfs.append(df)
df=pd.concat(coin_dfs, axis=1)
df=df.loc[collectDataFrom:]
df.sort_index(inplace=True)
df


Request : https://min-api.cryptocompare.com/data/v2/histoday?fsym=BTC&tsym=USD&limit=2000&api_key=777db0605f662ba6667a0c8d385d317262bd221764d4a3b01dc7a629e9592a49&toTs=1677627126.005913


MemoryError: Cannot allocate write+execute memory for ffi.callback(). You might be running on a system that prevents this. For more information, see https://cffi.readthedocs.io/en/latest/using.html#callbacks

## Narrowing Input and Refining Input Parameters

In [ ]:
# ----- User Settings Overrides
#coins=['BTC', 'ETH','LTC','MKR','AAVE', 'BAL', 'UNI', 'LDO', 'MATIC','LINK']
coins=['ETH','BTC','LTC']

trading_fee = 0.5/100 
trading_range = [+0.02,+2]
#buyAbove='sma7d'
#buyAbove='sma50d'
#buyAbove='sma200d'
buyAbove='sma200w'
# df=df.loc['2020-01-01':]

## Adding Extra Columns

In [ ]:
for coin in coins:
    
    # Adding Simple Moving Average Columns
    df.loc(axis=1)[(coin,'sma7d')] = df.loc(axis=1)[(coin,'close')].rolling(7).mean()
    df.loc(axis=1)[(coin,'sma50d')] = df.loc(axis=1)[(coin,'close')].rolling(50).mean()
    df.loc(axis=1)[(coin,'sma200d')] = df.loc(axis=1)[(coin,'close')].rolling(200).mean()
    df.loc(axis=1)[(coin,'sma200w')] = df.loc(axis=1)[(coin,'close')].rolling(200*7).mean()
    # Adding ExponentialMoving Average Columns
    #df.loc(axis=1)[(coin,'ema21w')] = df.loc(axis=1)[(coin,'close')].ewm(span=21*7, adjust=False).mean()
    
    # Setting Initial coins

    df.loc(axis=1)[(coin,'usd_bag')] = initial_usd;
    df.loc(axis=1)[(coin,'coin_bag')] = initial_coin;
df.sort_index(axis=1,inplace=True, level='Coin',sort_remaining=False)
df['ETH']




## Calculate Trading Strategy Results

In [ ]:


def trade_by_above_ma(row, ma_column='sma7d'):

    isAboveSMA = row['close'] > row[ma_column]
    coin_bag = row['coin_bag']
    usd_bag = row['usd_bag']
    if pd.isna(row['close']):
        return coin_bag, usd_bag 
    if isAboveSMA == True:  # Buy
        if usd_bag>0:
            coin_bag =  row['coin_bag'] + (1-trading_fee) * row['usd_bag']/row['close']  
            usd_bag = 0
    else:                   # Sell
        if coin_bag>0: 
            usd_bag = row['usd_bag'] + (1-trading_fee) * row['coin_bag']*row['close']
            coin_bag = 0
    return coin_bag, usd_bag 


def trade_by_ma_range(row, ma_column='sma7d', trading_range = [-0.1,+0.1] ):
    
    coin_bag = row['coin_bag']
    usd_bag = row['usd_bag']
    if pd.isna(row['close']):
        return coin_bag, usd_bag 
    if row['close'] < (1+trading_range[0])*row[ma_column] :    
        if usd_bag>0:  # Buy
            coin_bag =  row['coin_bag'] + (1-trading_fee) * row['usd_bag']/row['close']  
            usd_bag = 0
    else :
        if row['close'] > (1+trading_range[1])*row[ma_column] :
            if coin_bag>0: # Sell
                usd_bag = row['usd_bag'] + (1-trading_fee) * row['coin_bag']*row['close']
                coin_bag = 0
    return coin_bag, usd_bag 



for coin in coins:
    
    usd_bag=initial_usd
    coin_bag=initial_coin

    for index, row in df[coin].iterrows():

        row['coin_bag'] = coin_bag
        row['usd_bag'] = usd_bag
        #coin_bag, usd_bag = trade_by_above_ma(row, ma_column=buyAbove)
        coin_bag, usd_bag = trade_by_ma_range(row, ma_column=buyAbove, trading_range=trading_range )
        #df[coin].loc[index,'coin_bag'] = coin_bag
        df.loc[index,(coin,'coin_bag')] = coin_bag
        df.loc[index,(coin,'usd_bag')] = usd_bag
        #df[coin].loc[index,'usd_bag'] = usd_bag
    # df[coin]['usd_value'] = df[coin]['usd_bag'] + df[coin]['coin_bag']*df[coin]['close']
    df.loc(axis=1)[(coin,'trade_usd_value')] = df[coin]['usd_bag'] + df[coin]['coin_bag']*df[coin]['close']
    df.loc(axis=1)[(coin,'hodl_usd_value')] = df[coin]['close']*initial_coin + initial_usd
    df.loc(axis=1)[(coin,'trade_success_rate')] = df[coin]['trade_usd_value'] / df[coin]['hodl_usd_value'] 

df.sort_index(axis=1,inplace=True, level='Coin',sort_remaining=False)
df

## Showing Results

In [ ]:

import matplotlib.pyplot as plot

coin='ETH'
df[coin]['close'].plot(c='orange',label='Hodl Portfolio USD Value',linewidth=2)
df[coin][buyAbove].plot(c='red',label=buyAbove)
df[coin]['trade_usd_value'].plot(c='blue',label='Buy above {} strategy USD Value'.format(buyAbove),linewidth=2)


plot.title("{} Moving Average Trading Based on {}".format(coin, buyAbove), fontsize=20)
plot.ylabel("{} Price".format(coin))
plot.xlabel('Date')
plot.legend()
plot.yscale('log')
F = plot.gcf()

# Now check everything with the defaults:
#DPI = F.get_dpi()
#print ("DPI:", DPI)
DefaultSize = F.get_size_inches()
#print ("Default size in Inches", DefaultSize)
#print ("Which should result in a %i x %i Image"%(DPI*DefaultSize[0], DPI*DefaultSize[1]))
# the default is 100dpi for savefig:
#F.savefig("test1.png")
# this gives me a 797 x 566 pixel image, which is about 100 DPI

# Now make the image twice as big, while keeping the fonts and all the
# same size
F.set_size_inches( (DefaultSize[0]*3, DefaultSize[1]*2) )

In [ ]:

for coin in coins:
    df[coin]['trade_success_rate'].plot(c=coin_colors[coin],label="Better Than hodling {}".format(coin),linewidth=2)

plot.title("Moving Average Trading Based on {} vs Hodl Success Rate ".format(buyAbove), fontsize=20)
plot.ylabel("Portfolio success ratio over hodl")
plot.xlabel('Date')
plot.legend()
F = plot.gcf()
DefaultSize = F.get_size_inches()

F.set_size_inches( (DefaultSize[0]*3, DefaultSize[1]*2) )

In [ ]:
# Apply the default theme
import seaborn as sns
coin='BTC'
sns.set_theme( palette="pastel")
sns.set(rc = {'figure.figsize':(15,8)})

plot.title("{} Moving Average Trading Based on {}".format(coin, buyAbove), fontsize=20)
g_results=sns.lineplot(data=df[coin], x=df[coin].index, y=df[coin]['close'], color=coin_colors[coin],label='{} Price'.format(coin))
g_results=sns.lineplot(data=df[coin], x=df[coin].index, y=df[coin][buyAbove], color='red', label=buyAbove)
g_results=sns.lineplot(data=df[coin], x=df[coin].index, y=df[coin]['trade_usd_value'], color='blue', label='Trading Value {}'.format(buyAbove),linewidth=1)

g_results.set(yscale='log')
plot.legend()